# LGBM Model
* RNN was far too slow and hence infeasible given the time limit I am constrained within
* So going to use an LGBM

# Misc / setting up

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
from datetime import datetime
from IPython.display import display
from pprint import pprint
import lightgbm as lgbm
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, StratifiedKFold
import time

In C:\Users\psydm7\Anaconda3\envs\mle_tf\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\psydm7\Anaconda3\envs\mle_tf\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\psydm7\Anaconda3\envs\mle_tf\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\psydm7\Anaconda3\envs\mle_tf\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\psydm7\Anaconda3\envs\mle_tf

In [3]:
SEED = 137

In [ ]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

# loading the data

In [ ]:
df_train = pd.read_pickle("../Data/Prepared_Data/supervised/consumption_all.pkl")
display(df_train)
df_preds = pd.read_pickle("../Data/Prepared_Data/supervised/consumption_all_preds.pkl")
display(df_preds)

# formatting the df's

In [ ]:
#replacing nans with 0 so we can aggregate up the OOF predictions
df_preds["meter_reading"] = df_preds["meter_reading"].fillna(0) 
#dropping the "energy n-k" columns as they are needed for 3D RNN input not 2D LGBM input
df_preds = df_preds.drop(["energy n-7","energy n-6","energy n-5","energy n-4","energy n-3","energy n-2","energy n-1"], axis=1)
df_train = df_train.drop(["energy n-7","energy n-6","energy n-5","energy n-4","energy n-3","energy n-2","energy n-1"], axis=1)

display(df_train)
display(df_preds)

# Ordinally encoding the ID's
* One hot / binary encoding can actually worsen performance of DT based algorithms
* This is a LGBM so a gradient boosted decision tree
* Hence I will also now encode the ID ordinally and experiment with both to see which gives the best performing results

In [ ]:
#ordinally encoding id's
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df_train["meter_id"])

#ordinally encoding with same encoder so will encode the same
df_train.insert(1, "meter_id_ord", le.transform(df_train["meter_id"]))
df_preds.insert(1, "meter_id_ord", le.transform(df_preds["meter_id"]))

display(df_train)
display(df_preds)

# Defining LGBM model hyperparameters
* Will need to do those multiple times 
* Cannot automate this as don't have access to the test data will have to just submit multiple times and check
* Will have to do manual hyper parameter tuning

## Defining the columns we will use
* Multiple configurations to test during manual hyper parameter tuning

In [ ]:
y_col = "meter_reading" #we want to predict the meter_reading

# all consumption based variables, no addInfo or cluster, meter_id ordinally encoded
# score: 
X_cols = ["meter_id_ord","day_of_year_sin","day_of_year_cos","day_of_week","day_of_month","month_ord","month_sin","month_cos","is_weekend"]
X_cat = ["meter_id_ord","day_of_week","day_of_month","month_ord","is_weekend"]

# description
# score: 
#X = []
#X_cat = []

In [ ]:
# setting categorical columns in the dataframe to be categorical
for i in X_cat:
    df_train[i] = df_train[i].astype('category')
    df_preds[i] = df_preds[i].astype('category')

In [ ]:
display(df_train[X_cols].head(3))
display(df_preds[X_cols].head(3))

## Defining the model parameters
* also will need to be manually hyper tuned after submitting each combination to the comp and getting results

In [ ]:
params = {
    'boosting_type': 'gbdt', #gbdt/rf/dart/goss
    'metric': 'mae',
    'num_threads': 2, # number of threads to run on for speed
    
    'max_leaf': 256, # limit max numer of leaves in a tree
    "max_depth":10, # limit max depth of the tree to prevent overfitting
    
    #defining the models runs
    'learning_rate': 0.01,
    'num_iterations': 10000,
    
    # fraction to be bagged/sampled every k iterations
    'bagging_fraction': 0.85,
    'bagging_freq' : 10,
    
    'feature_fraction': 0.85, # fraction of features to use at each tree node
    
    #l1 & l2 regularization to prevent overfitting
    "lambda_l1": 16,
    "lambda_l2": 8,

    'seed': SEED # all runs with same seed for better comparison between different hyper params
    
}

pprint(params)

# Running LGBM with SKF
* Using skf on df_train by meter_id with 3 folds
    * Meaning for each iteration we use 2/3 of each meters data for training and 1/3 of each meters data for validating
* using out of fold predictions, making predictions on each fold and aggregating them together for the final prediction
# WHICH IS BETTER?
## CALCULATING PREDICTIONS AND RETURNING THEM

In [ ]:
import warnings
warnings.filterwarnings("ignore")

#function for running the cross fold
    #args:
        #print_bool = True if we want to print info for each fold
        #SEED = random seed used for fair repeatability
        #num_folds = number of folds in skf (pretty certain going to keep this at 3 )
        #df_train = training dataframe
        #df_preds = preds dataframe we are going to populate
        #y_col = the name of the label we want to predict (meter_reading)
        #X_cols = the name of the feature columns we are using
        #X_cat = the name of these features which are categorical
        #params = hyper params for the LGBM model
    #returns:
        #time_of_execution = how long it took to train the model on all folds; will be used as a point of comparison
        #valid_score = MAE calculated using the Out-of-Fold Predictions on the df_train, used for hyper-param tuning
        #df_train = the training set containing the OOF column (returning so can inspect if I wish)
        #df_preds = the prediction set now populated with the average forecast of the folds
        
def run_lgbm_skf_cv(print_bool, SEED, num_folds, df_train, df_preds, y_col, X_cols, X_cat, params):
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = SEED) # defining the SKF algorithm

    start_time = time.time()
    fold_iter=1
    #running the startified kfold, splitting df_train by meter_id, so we use 2/3 of each meters reading for training
    for train_index, valid_index in skf.split(df_train, df_train["meter_id"]):

        print(f"{color.BOLD}{color.BLUE}{color.UNDERLINE}Fold {fold_iter}{color.END}")
        
        #splitting into the features and labels for the train and valid folds
        X_train, X_valid = df_train.loc[train_index, X_cols], df_train.loc[valid_index, X_cols]
        y_train, y_valid = df_train.loc[train_index, y_col], df_train.loc[valid_index, y_col]
        
        if(print_bool):
            print(f"{color.CYAN}{color.UNDERLINE}X_train{color.END}")
            display(X_train.head(5))
            print(f"{color.CYAN}{color.UNDERLINE}y_train{color.END}")
            display(y_train.head(5))
            print(f"{color.CYAN}{color.UNDERLINE}X_valid{color.END}")
            display(X_valid.head(5))
            print(f"{color.CYAN}{color.UNDERLINE}y_valid{color.END}")
            display(y_valid.head(5))
            
        print(f"{color.CYAN}{color.UNDERLINE}Training the LGBM{color.END}")
        #instantiating a lgbm regressor with our params
        lgbm_model = lgbm.LGBMRegressor(**params)
        #fitting the lgbm model on the 2/3 train and evaluating on the 1/3 valid
        #printing details every 1000 iters + stopping if no improvement made in 250 iters
        lgbm_model.fit(X_train, y_train,
                       eval_set=[(X_valid, y_valid)],
                       categorical_feature=X_cat,
                       verbose=1000,
                       early_stopping_rounds=250)
        
        #saving the OOF prediction for the held out rows (valid rows from df_train) from the lgbm model with the best performing intrinisic parmams 
        oof_valid = lgbm_model.predict(X_valid, num_iteration=lgbm_model.best_iteration_) # making prediction on the held out rows, X_valid
        df_train.loc[valid_index, "oof"] = oof_valid #storing the oof rows 
        
        #calculating the average preds by summing the weighted preds for each fold
        pred_predictions = lgbm_model.predict(df_preds[X_cols], num_iteration=lgbm_model.best_iteration_) #predicting the unkown df_preds
        df_preds[y_col] += pred_predictions / num_folds #weighting the predictions for this fold and adding to df_preds y column 
        
        if(print_bool):
            print(f"{color.CYAN}{color.UNDERLINE}df_train OOF predictions{color.END}")
            display(df_train.loc[valid_index, "oof"].head(5))
            print(f"{color.CYAN}{color.UNDERLINE}this folds df_preds OOF predictions{color.END}")
            display(pred_predictions[:5])
            print(f"{color.CYAN}{color.UNDERLINE}weighted aggregated df_preds OOF predictions so far{color.END}")
            pprint(df_preds[y_col].head(5))
               
        fold_iter+=1
        
    df_preds[y_col]=df_preds.meter_reading.clip(lower=0)#clipping the pred results in case any predictions were below 0 
    #calculating execution time and the MAE on the training set
    time_of_execution = time.time() - start_time
    valid_score=mean_absolute_error(df_train[y_col], df_train["oof"]) 
    return (time_of_execution, valid_score, df_train, df_preds)

In [ ]:
#running lgbm with cv skf and storing the results
results = run_lgbm_skf_cv(False, SEED, 3, df_train.copy(), df_preds.copy(), y_col, X_cols, X_cat, params)
LGBM_TOE = results[0]
LGBM_MAE = results[1]
LGBM_DF_TRAIN = results[2]
LGBM_DF_PREDS = results[3]

In [ ]:
#displaying the results
print(f"{color.BOLD}{color.CYAN}{color.UNDERLINE}Time of execution{color.END}: {LGBM_TOE} seconds")
print(f"{color.BOLD}{color.CYAN}{color.UNDERLINE}Mean absolute error between OOF preds and meter_reading on training set{color.END}: {LGBM_MAE}")
print(f"{color.BOLD}{color.CYAN}{color.UNDERLINE}training set OOF preds vs true{color.END}")
display(LGBM_DF_TRAIN[["meter_id","date","meter_reading","oof"]])
print(f"{color.BOLD}{color.CYAN}{color.UNDERLINE}preds set with the next years forecasts for each meter{color.END}")
display(LGBM_DF_PREDS[["meter_id","date","meter_reading"]])

## OR RETURN LGBMS AND CALCULATE PREDICTIONS AFTER?

In [ ]:
import warnings
warnings.filterwarnings("ignore")

#function for running the cross fold
    #args:
        #print_bool = True if we want to print info for each fold
        #SEED = random seed used for fair repeatability
        #num_folds = number of folds in skf (pretty certain going to keep this at 3 )
        #df_train = training dataframe
        #y_col = the name of the label we want to predict (meter_reading)
        #X_cols = the name of the feature columns we are using
        #X_cat = the name of these features which are categorical
        #params = hyper params for the LGBM model
    #returns:
        #time_of_execution = how long it took to train the model on all folds; will be used as a point of comparison
        #valid_score = MAE calculated using the Out-of-Fold Predictions on the df_train, used for hyper-param tuning
        #df_train = the training set containing the OOF column (returning so can inspect if I wish)
        #lgbm_models = array of the different lgbm_models
        
def run_lgbm_skf_cv(print_bool, SEED, num_folds, df_train, y_col, X_cols, X_cat, params):
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = SEED) # defining the SKF algorithm

    lgbm_models = []
    start_time = time.time()
    fold_iter=1
    #running the startified kfold, splitting df_train by meter_id, so we use 2/3 of each meters reading for training
    for train_index, valid_index in skf.split(df_train, df_train["meter_id"]):

        print(f"{color.BOLD}{color.BLUE}{color.UNDERLINE}Fold {fold_iter}{color.END}")
        
        #splitting into the features and labels for the train and valid folds
        X_train, X_valid = df_train.loc[train_index, X_cols], df_train.loc[valid_index, X_cols]
        y_train, y_valid = df_train.loc[train_index, y_col], df_train.loc[valid_index, y_col]
        
        if(print_bool):
            print(f"{color.CYAN}{color.UNDERLINE}X_train{color.END}")
            display(X_train.head(5))
            print(f"{color.CYAN}{color.UNDERLINE}y_train{color.END}")
            display(y_train.head(5))
            print(f"{color.CYAN}{color.UNDERLINE}X_valid{color.END}")
            display(X_valid.head(5))
            print(f"{color.CYAN}{color.UNDERLINE}y_valid{color.END}")
            display(y_valid.head(5))
            
        print(f"{color.CYAN}{color.UNDERLINE}Training the LGBM{color.END}")
        #instantiating a lgbm regressor with our params
        lgbm_model = lgbm.LGBMRegressor(**params)
        #fitting the lgbm model on the 2/3 train and evaluating on the 1/3 valid
        #printing details every 1000 iters + stopping if no improvement made in 250 iters
        lgbm_model.fit(X_train, y_train,
                       eval_set=[(X_valid, y_valid)],
                       categorical_feature=X_cat,
                       verbose=1000,
                       early_stopping_rounds=250)
        
        #saving the OOF prediction for the held out rows (valid rows from df_train) from the lgbm model with the best performing intrinisic parmams 
        oof_valid = lgbm_model.predict(X_valid, num_iteration=lgbm_model.best_iteration_) # making prediction on the held out rows, X_valid
        df_train.loc[valid_index, "oof"] = oof_valid #storing the oof rows 
        if(print_bool):
            print(f"{color.CYAN}{color.UNDERLINE}df_train OOF predictions{color.END}")
            display(df_train.loc[valid_index, ["meter_id","oof"]].head(5))
               
        #appending this lgbm
        lgbm_models.append(lgbm_model)
        if(print_bool):
            print(f"{color.CYAN}{color.UNDERLINE}lgbm_models size{color.END}: {len(lgbm_models)}")
            
        fold_iter+=1
        
    #calculating execution time and the MAE on the training set
    time_of_execution = time.time() - start_time
    valid_score=mean_absolute_error(df_train[y_col], df_train["oof"]) 
    return (time_of_execution, valid_score, df_train, lgbm_models)


In [ ]:
#running lgbm with cv skf and storing the results
results = run_lgbm_skf_cv(False, SEED, 3, df_train.copy(), y_col, X_cols, X_cat, params)
LGBM_TOE = results[0]
LGBM_MAE = results[1]
LGBM_DF_TRAIN = results[2]
LGBM_MODELS = results[3]

In [ ]:
#only make predictions on the best performing LGBM MODELS (one returned by grid search)
LGBM_DF_PREDS = df_preds.copy()

#calculating the average preds by summing the weighted preds for each fold
for lgbm_i in range(len(LGBM_MODELS)):
    pred_predictions = lgbm_model.predict(LGBM_DF_PREDS[X_cols], num_iteration=lgbm_model.best_iteration_) #predicting the unkown df_preds
    LGBM_DF_PREDS[y_col] += pred_predictions / num_folds #weighting the predictions for this fold and adding to df_preds y column 

In [ ]:
#displaying the results
print(f"{color.BOLD}{color.CYAN}{color.UNDERLINE}Time of execution{color.END}: {LGBM_TOE} seconds")
print(f"{color.BOLD}{color.CYAN}{color.UNDERLINE}Mean absolute error between OOF preds and meter_reading on training set{color.END}: {LGBM_MAE}")
print(f"{color.BOLD}{color.CYAN}{color.UNDERLINE}training set OOF preds vs true{color.END}")
display(LGBM_DF_TRAIN[["meter_id","date","meter_reading","oof"]])
print(f"{color.BOLD}{color.CYAN}{color.UNDERLINE}preds set with the next years forecasts for each meter{color.END}")
display(LGBM_DF_PREDS[["meter_id","date","meter_reading"]])